In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
!pip install -Uqq sarvamai
from sarvamai import SarvamAI

In [3]:
SARVAM_API_KEY = "0140c592-6272-4436-8398-111390895f78"
client = SarvamAI(api_subscription_key=SARVAM_API_KEY)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of India?"},
]

response = client.chat.completions(
    messages=messages,
)

# Extract and print the assistant's reply
reply = response.choices[0].message.content
print("Response:", reply)

Response:  The capital of India is **New Delhi**. It serves as the political and administrative center of the country, housing key government institutions such as the Parliament, the Rashtrapati Bhavan (President's residence), and the Supreme Court. New Delhi is part of the larger National Capital Region (NCR) and has a rich history, blending modern infrastructure with historical landmarks like the Red Fort and Qutub Minar.
